# Proyecto: Persistencia de información

La persistencia se refiere al respaldo de información (en disco o memoria) de un Data Frame o RDD con el objetivo de evitar pérdidas de información.

Algunas de las funciones que veremos son:

`.is_cached` <-- Verifica si un Data Frame o RDD esta en memoria computacional 

`.rdd.cache()` <-- Guarda un rdd en memoria

`.rdd.getStorageLevel()` <-- Verifica las caracteriasticas de persistencia

`.rdd.unpersist()` <-- Elimina la persistenca de información

`.rdd.persist()` <-- Aplica persistenca de información

In [2]:
# libreria para crear punto de conexión:
from pyspark import SparkContext

# Cargamos libreria para crear Data Frames:
from pyspark.sql import SQLContext

#from pyspark.sql import SparkSession

# Cargamos librerias para crear el schema del DataFrame
from pyspark.sql.types import StructType, StructField

# Cargamos los tipos de datos que usaremos para crear las columnas de los dataframes:
from pyspark.sql.types import  IntegerType, StringType, FloatType

#from pyspark.sql.types import Row

# Importamos functions especiales, entre ellas la función 'col'
from pyspark.sql.functions import *

# Importamos libreria para peristencia:
from pyspark.storagelevel import StorageLevel

In [3]:
# Creamos el punto de conexió a Spark (que se ejecutará en mi máquina 'local'):
spark = SparkContext(master='local', appName='Persistencia_Particionado')

In [4]:
# Creamos el contexto para SQL:
sqlContext = SQLContext(spark)

/home/marck/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# Visualizamos el contenido de la carpeta 'Data' 
!ls ./Data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [6]:
# Ruta en donde se encuentran los datos con los que trabajaremos:
path = './Data/'

In [7]:
# Creamos función para eliminar encabezados en RDDs:
def eliminaEncabezado(indice , interador):
    return iter( list(interador)[1:] )

In [8]:
# Número de registros a visualizar:
N=5

## Creación de un DataFrame con los datos del archivo *paises.csv*

In [9]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/paises.csv

id,equipo,sigla
1,30. Februar,AUT
2,A North American Team,MEX
3,Acipactli,MEX
4,Acturus,ARG


In [10]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
paisesSchema = StructType([
    StructField('id',IntegerType(),False),
    StructField('equipo',StringType(),False),
    StructField('sigla',StringType(),False) 
])

# Creamos el DataFrame:
paisesDF = sqlContext.read.schema(paisesSchema).option('header', 'true').csv(path+'paises.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
paisesDF.show(N)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *deporte.csv*

In [11]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deporte.csv

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War


In [12]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
deportesSchema = StructType([
    StructField('deporte_id',IntegerType(),False),
    StructField('deporte',StringType(),False) 
])

# Creamos el DataFrame:
deportesDF = sqlContext.read.schema(deportesSchema).option('header', 'true').csv(path+'deporte.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportesDF.show(N)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *evento.csv*

In [13]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/evento.csv

evento_id,evento,deporte_id
1,Basketball Men's Basketball,1
2,Judo Men's Extra-Lightweight,2
3,Football Men's Football,3
4,Tug-Of-War Men's Tug-Of-War,4


In [14]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
eventoSchema = StructType([
    StructField('evento_id',IntegerType(),False),
    StructField('nombre',StringType(),False),
    StructField('deporte_id',IntegerType(),False) 
])

# Creamos el DataFrame:
deportesOlimpicosDF = sqlContext.read.schema(eventoSchema).option('header', 'true').csv(path+'evento.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportesOlimpicosDF.show(N)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *resultados.csv*

In [15]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/resultados.csv

resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4


In [16]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
resultadoSchema = StructType([
    StructField('resultado_id',IntegerType(),False),
    StructField('medalla',StringType(),False),
    StructField('deportista_id',IntegerType(),False),
    StructField('juego_id',IntegerType(),False),
    StructField('evento_id',IntegerType(),False)
])

# Creamos el DataFrame:
resultadoDF = sqlContext.read.schema(resultadoSchema).option('header', 'true').csv(path+'resultados.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
resultadoDF.show(N)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *juegos.csv*

In [17]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina


In [18]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
juegoSchema = StructType([
    StructField('juego_id',IntegerType(),False),
    StructField('nombre_juego',StringType(),False),
    StructField('anio',IntegerType(),False),
    StructField('temporada',StringType(),False),
    StructField('ciudad',StringType(),False)
])

# Creamos el DataFrame:
juegoDF = sqlContext.read.schema(juegoSchema).option('header', 'true').csv(path+'juegos.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
juegoDF.show(N)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego|anio|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



In [19]:
# Eliminamos la una columna del Data Frame:
juegoDF = juegoDF.drop('nombre_juego')

juegoDF.show(N)

+--------+----+---------+---------+
|juego_id|anio|temporada|   ciudad|
+--------+----+---------+---------+
|       1|1896|   Verano|   Athina|
|       2|1900|   Verano|    Paris|
|       3|1904|   Verano|St. Louis|
|       4|1906|   Verano|   Athina|
|       5|1908|   Verano|   London|
+--------+----+---------+---------+
only showing top 5 rows



## Creación de un DataFrame con los datos de los archivos *deportista.csv* y *deportista2.csv*

In [20]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deportista.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,0,0,273
4,Edgar Lindenau Aabye,1,34,0,0,278


In [21]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deportista2.csv

67787,Lee BongJu,1,27,167,56,970
67788,Lee BuTi,1,23,164,54,203
67789,Anthony N. Buddy Lee,1,34,172,62,1096
67790,Alfred A. Butch Lee Porter,1,19,186,80,825
67791,Lee ByeongGu,1,22,175,68,970


Notemos que el archivo 'deportista.csv' tiene encabezado y 'deportista2.csv' no tiene

El archivo 'deportista2.csv' es continuación de 'deportista.csv'

Para hacer un sólo Data Frame, haremos lo siguiente: 

1. Tranformamos los archivos.csv a RDDs

2. Eliminamos el encabezado del archivo 'deportista.csv'

3. Unimos los RDDs 'deportista' y 'deportista2

4. Ya que tenemos un sólo RDD lo pasamos a un Data Frame

In [22]:
# Guardamos archivos.csv en RDDs:
# .map(lambda l : l.split(',')) <-- Le asigan formato al contenido de archivos .csv

deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l : l.split(','))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l : l.split(','))

In [23]:
# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [24]:
# Eliminamos encabezado al primer RDD :
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [25]:
# Vemos los primeros N registros del segundo RDD:
deportistaOlimpicoRDD2.take(N)

[['67787', 'Lee BongJu', '1', '27', '167', '56', '970'],
 ['67788', 'Lee BuTi', '1', '23', '164', '54', '203'],
 ['67789', 'Anthony N. Buddy Lee', '1', '34', '172', '62', '1096'],
 ['67790', 'Alfred A. Butch Lee Porter', '1', '19', '186', '80', '825'],
 ['67791', 'Lee ByeongGu', '1', '22', '175', '68', '970']]

In [26]:
# Unimos los 2 RDDs: deportistaOlimpicoRDD y deportistaOlimpicoRDD2
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union( deportistaOlimpicoRDD2 )

In [27]:
# Creación de un Data Frame a partir de un RDD:

# Corregimos los tipos de dato en cada columna del RDD: 
deportistaOlimpicoRDD = \
    deportistaOlimpicoRDD.map(lambda x: ( int(x[0]), x[1],  int(x[2]), int(x[3]), int(x[4]), float(x[5]), int(x[6])  ) )


# A continuación creamos el schema, es decir, la estructura (columnas) del Data Frame donde pondremos los datos del RDD:
schema = StructType([
    StructField('deportista_id', IntegerType(), False ),
    StructField('nombre', StringType(), False ),
    StructField('genero', IntegerType(), False ),
    StructField('edad', IntegerType(), False ),
    StructField('altura', IntegerType(), False ),
    StructField('peso', FloatType(), False ),
    StructField('equipo_id', IntegerType(), False )
    ])

# StructField <-- instrucción para crear una columna, se debe especificar el tipo de dato que almacenará
# En caso de que el campo pueda ser nulo ponemos 'True' de lo contrario ponemos 'False'


# Creamos el DataFrame a partir del RDD:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)


# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportistaOlimpicoDF.show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [28]:
# Vemos los primeros N registros ordenadas de forma ascendente 
# con respecto a los valores de la columna 'edadAlJugar'

deportistaOlimpicoDF.sort('edad').show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|          133|           Franz Abb|     1|   0|     0| 0.0|      399|
|          167|Ould Lamine Abdallah|     1|   0|     0| 0.0|      362|
|           66|     Mohamed Abakkar|     1|   0|   156|48.0|     1003|
|          163|     Ismail Abdallah|     1|   0|     0| 0.0|     1095|
|          139|George Ioannis Abbot|     1|   0|     0| 0.0|     1043|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [29]:
# Nos quedamos con registros donde los valores de la columna 'edad' sean distintos de cero
deportistaOlimpicoDF = deportistaOlimpicoDF.filter( (deportistaOlimpicoDF['edad'] !=0) )

In [30]:
# Mostramos los primeros registros ordenados con respecto a los valores de la columna 'edad'
deportistaOlimpicoDF.sort('edad').show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|        71691|  Dimitrios Loundras|     1|  10|     0| 0.0|      333|
|        52070|        Etsuko Inada|     2|  11|     0| 0.0|      514|
|        40129|    Luigina Giavotti|     2|  11|     0| 0.0|      507|
|        37333|Carlos Bienvenido...|     1|  11|     0| 0.0|      982|
|        47618|Sonja Henie Toppi...|     2|  11|   155|45.0|      742|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



## Join entre 6 DataFrames

In [31]:
medallistaXAnio = deportistaOlimpicoDF.join(
        resultadoDF, 
        deportistaOlimpicoDF['deportista_id'] == resultadoDF['deportista_id'], 
        'left'
    ).join(
        juegoDF, 
        juegoDF['juego_id'] == resultadoDF['juego_id'], 
        'left'
    ).join(
        paisesDF, 
        deportistaOlimpicoDF['equipo_id'] == paisesDF['id'], 
        'left' 
    ).join(
        deportesOlimpicosDF, 
        deportesOlimpicosDF['evento_id'] == resultadoDF['evento_id'], 
        'left' 
    ).join(
        deportesDF, 
        deportesOlimpicosDF['deporte_id'] == deportesDF['deporte_id'], 
        'left' 
    ).select( #<-- seleccionamos y renombramos algunas columnas
        'sigla',
        'anio',
        'medalla',
        deportesOlimpicosDF['nombre'].alias('Nombre subdisciplina'),
        deportesDF['deporte'].alias('Nombre disciplina'),
        deportistaOlimpicoDF['nombre']    
    )

medallistaXAnio.show()

+-----+----+-------+--------------------+--------------------+--------------------+
|sigla|anio|medalla|Nombre subdisciplina|   Nombre disciplina|              nombre|
+-----+----+-------+--------------------+--------------------+--------------------+
|  CHN|1992|     NA|Basketball Men's ...|          Basketball|           A Dijiang|
|  CHN|2012|     NA|Judo Men's Extra-...|                Judo|            A Lamusi|
|  DEN|1920|     NA|Football Men's Fo...|            Football| Gunnar Nielsen Aaby|
|  SWE|1900|   Gold|Tug-Of-War Men's ...|          Tug-Of-War|Edgar Lindenau Aabye|
|  NED|1994|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1994|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1992|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1992|     NA|Speed Skating Wom...|       Speed Skating|Christine Jacoba ...|
|  NED|1988|     NA|Speed Skating Wom...|       Speed Skating|Christine Jaco

In [32]:
# Filtramos los datos quedándonos con los deportistas que sí tienen medallas: 
medallistaXAnio.filter( medallistaXAnio['medalla'] != 'NA'  )\
    .groupBy('sigla','anio','Nombre subdisciplina')\
    .count()\
    .sort('anio')\
    .show()

+-----+----+--------------------+-----+
|sigla|anio|Nombre subdisciplina|count|
+-----+----+--------------------+-----+
|  GRE|1896|Athletics Men's 8...|    1|
|  AUS|1896|Athletics Men's 1...|    1|
|  GBR|1896|Tennis Men's Singles|    1|
|  USA|1896|Athletics Men's 1...|    2|
|  AUT|1896|Swimming Men's 10...|    1|
|  AUS|1896|Tennis Men's Doubles|    1|
|  HUN|1896|Athletics Men's M...|    1|
|  GBR|1896|Weightlifting Men...|    1|
|  HUN|1896|Swimming Men's 10...|    1|
|  USA|1896|Athletics Men's 1...|    1|
|  GER|1896|Athletics Men's 1...|    1|
|  GRE|1896|Tennis Men's Singles|    2|
|  GER|1896|Cycling Men's Roa...|    1|
|  GBR|1896|Athletics Men's 4...|    1|
|  FRA|1896|Fencing Men's Foi...|    2|
|  GRE|1896|Tennis Men's Doubles|    2|
|  GER|1896|Gymnastics Men's ...|    9|
|  GBR|1896|Cycling Men's 12-...|    1|
|  GER|1896|Gymnastics Men's ...|    1|
|  USA|1896|Athletics Men's 1...|    1|
+-----+----+--------------------+-----+
only showing top 20 rows



## Verificamos si el Data Frame esta en memoria:

In [35]:
# preguntamos si el Data Frame esta en memoria:
medallistaXAnio.is_cached

False

Vemos que el DF no esta en memoria, así que cada vez que lo usemos Spark tendrá que recomputarlo, para evitar esto hacemos lo mandamos a cache:

## Guardamos el Data Frame en cache:

In [61]:
# Si vamos a aplicar una persistencia, primero debemos quitar las persistencias previas (en caso de que se hayan hecho antes):
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[117] at javaToPython at NativeMethodAccessorImpl.java:0

In [62]:
# Aplicamos peristencia de información en 'cache'
medallistaXAnio.rdd.cache()

MapPartitionsRDD[117] at javaToPython at NativeMethodAccessorImpl.java:0

In [63]:
# Obtenemos las características del almacenamiento:
medallistaXAnio.rdd.getStorageLevel()

# 'StorageLevel(False, True, False, False, 1)' <-- indica las características de la persistencia

StorageLevel(False, True, False, False, 1)

## Aplicación de persistencia:

Documentación de los parámetros de la función `StorageLevel()` [aquí]( https://spark.apache.org/docs/2.4.6/api/python/pyspark.html#pyspark.StorageLevel )

`StorageLevel(useDisk, useMemory, useOffHeap, deserialized, replication=1) `


DISK_ONLY = StorageLevel(True, False, False, False, 1)

DISK_ONLY_2 = StorageLevel(True, False, False, False, 2)

MEMORY_AND_DISK = StorageLevel(True, True, False, False, 1)

MEMORY_AND_DISK_2 = StorageLevel(True, True, False, False, 2)

MEMORY_AND_DISK_SER = StorageLevel(True, True, False, False, 1)

MEMORY_AND_DISK_SER_2 = StorageLevel(True, True, False, False, 2)

MEMORY_ONLY = StorageLevel(False, True, False, False, 1)

MEMORY_ONLY_2 = StorageLevel(False, True, False, False, 2)

MEMORY_ONLY_SER = StorageLevel(False, True, False, False, 1)

MEMORY_ONLY_SER_2 = StorageLevel(False, True, False, False, 2)¶

OFF_HEAP = StorageLevel(True, True, True, False, 1)

**Los números 1 y 2 al final de las instrucciones, indican el número de replicas de información**



In [64]:
# Si vamos a aplicar una persistencia, primero debemos quitar las persistencias previas:
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[117] at javaToPython at NativeMethodAccessorImpl.java:0

In [65]:
# Creamos una persistencia de infomración:
medallistaXAnio.rdd.persist( StorageLevel.MEMORY_AND_DISK_2 )

# En caso de error, debemos quitar las persistencias previas ejecutando antes:
# medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[117] at javaToPython at NativeMethodAccessorImpl.java:0

## Creando un nivel de persistencia definido por el usuario:

In [66]:
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True,True,False,False,3)

# MEMORY_AND_DISK_3 <-- nombre asignado por el usuario

In [67]:
# Si vamos a aplicar una persistencia, primero debemos quitar las persistencias previas:
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[117] at javaToPython at NativeMethodAccessorImpl.java:0

In [68]:
# Aplicamos nuestro nivel de persistencia:
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[117] at javaToPython at NativeMethodAccessorImpl.java:0

In [ ]:
# Cerramos sesión para liberar memoria:
spark.stop()